In [ ]:
import pandas as pd
from datetime import datetime
import os

# Constantes globales
INVENTARIO_FILE = "Inventario_Tienda_Barrio (1).xlsx"
VENTAS_FILE = "ventas_registradas.xlsx"
COLUMNAS_INVENTARIO = ['Categoría', 'Nombre', 'Código', 'Precio Unitario', 'Cantidad en Stock']

# Cargar o inicializar datos (las funciones anteriores se mantienen igual)
# ... [las funciones de carga de datos permanecen iguales] ...

# Funciones mejoradas de registro de ventas
def registrar_venta(inventario, ventas):
    """Registra una nueva venta con subtotales y opciones de finalización"""
    print("\n=== REGISTRAR VENTA ===")
    productos_vendidos = []
    cantidades_vendidas = []
    precios_unitarios = []
    subtotales = []
    total = 0

    while True:
        mostrar_productos(inventario)
        print("\n--- RESUMEN DE VENTA ACTUAL ---")
        if productos_vendidos:
            mostrar_resumen_venta(productos_vendidos, cantidades_vendidas, precios_unitarios, subtotales, total)
        else:
            print("No hay productos agregados aún.")

        print("\nOpciones:")
        print("1. Agregar producto")
        print("2. Terminar compra")
        print("3. Cancelar compra")
        opcion = input("Seleccione una opción: ")

        if opcion == "2":
            if total > 0:
                return finalizar_venta(inventario, ventas, productos_vendidos, cantidades_vendidas, precios_unitarios, subtotales, total)
            else:
                print("No hay productos para registrar. Venta cancelada.")
                return inventario, ventas
        elif opcion == "3":
            print("Venta cancelada. No se han realizado cambios.")
            return inventario, ventas
        elif opcion != "1":
            print("Opción no válida. Intente nuevamente.")
            continue

        # Agregar producto
        codigo = input("\nIngrese el código del producto vendido: ")

        if codigo.lower() == 'fin':
            continue

        if codigo not in inventario['Código'].values:
            print("Código no válido. Intente nuevamente.")
            continue

        producto = inventario[inventario['Código'] == codigo].iloc[0]
        cantidad_disponible = producto['Cantidad en Stock']

        try:
            cantidad = int(input(f"Cantidad de '{producto['Nombre']}' a vender (disponible: {cantidad_disponible}): "))

            if cantidad <= 0:
                print("La cantidad debe ser mayor a cero.")
                continue

            if cantidad > cantidad_disponible:
                print("No hay suficiente stock.")
                continue

            # Calcular subtotal
            subtotal = producto['Precio Unitario'] * cantidad

            # Agregar a la venta actual
            productos_vendidos.append(producto['Nombre'])
            cantidades_vendidas.append(cantidad)
            precios_unitarios.append(producto['Precio Unitario'])
            subtotales.append(subtotal)
            total += subtotal

            print(f"\nAgregado: {cantidad} x {producto['Nombre']} = ${subtotal:.2f}")

        except ValueError:
            print("Ingrese un número válido.")
            continue

def mostrar_resumen_venta(productos, cantidades, precios, subtotales, total):
    """Muestra un resumen de la venta en curso"""
    print("\nDetalle de la venta:")
    print("-" * 50)
    print(f"{'Producto':<20} {'Cant.':>5} {'P.Unit.':>10} {'Subtotal':>10}")
    print("-" * 50)

    for prod, cant, precio, subt in zip(productos, cantidades, precios, subtotales):
        print(f"{prod:<20} {cant:>5} {precio:>10.2f} {subt:>10.2f}")

    print("-" * 50)
    print(f"{'TOTAL:':>36} {total:>10.2f}")
    print("-" * 50)

def finalizar_venta(inventario, ventas, productos, cantidades, precios, subtotales, total):
    """Finaliza la venta actualizando inventario y registrando la venta"""
    # Actualizar inventario
    for producto, cantidad in zip(productos, cantidades):
        codigo = inventario[inventario['Nombre'] == producto].iloc[0]['Código']
        idx = inventario[inventario['Código'] == codigo].index[0]
        inventario.loc[idx, 'Cantidad en Stock'] -= cantidad

    # Guardar venta
    fecha_actual = datetime.now().strftime("%Y-%m-%d")
    hora_actual = datetime.now().strftime("%H:%M:%S")

    nueva_venta = pd.DataFrame([[fecha_actual, hora_actual,
                               ", ".join(productos),
                               ", ".join(map(str, cantidades)),
                               ", ".join(map(str, precios)),
                               ", ".join(map(lambda x: f"{x:.2f}", subtotales)),
                               total]],
                             columns=['Fecha', 'Hora', 'Productos', 'Cantidades', 'Precios', 'Subtotales', 'Total'])

    ventas = pd.concat([ventas, nueva_venta], ignore_index=True)
    print(f"\nVenta registrada con éxito! Total: ${total:.2f}")

    # Generar factura detallada
    generar_factura_detallada(fecha_actual, hora_actual, productos, cantidades, precios, subtotales, total)

    return inventario, ventas

def generar_factura_detallada(fecha, hora, productos, cantidades, precios, subtotales, total):
    """Genera una factura detallada en formato de texto"""
    print("\n" + "=" * 50)
    print("FACTURA DE VENTA".center(50))
    print("=" * 50)
    print(f"Fecha: {fecha} {hora}")
    print("-" * 50)
    print(f"{'Producto':<20} {'Cant.':>5} {'P.Unit.':>10} {'Subtotal':>10}")
    print("-" * 50)

    for prod, cant, precio, subt in zip(productos, cantidades, precios, subtotales):
        print(f"{prod:<20} {cant:>5} {precio:>10.2f} {subt:>10.2f}")

    print("-" * 50)
    print(f"{'TOTAL:':>36} {total:>10.2f}")
    print("=" * 50)
    print("¡GRACIAS POR SU COMPRA!".center(50))
    print("=" * 50 + "\n")

# Las demás funciones del sistema permanecen iguales
# ... [resto del código permanece igual] ...

if __name__ == "__main__":
    main()


=== SISTEMA DE GESTIÓN DE TIENDA ===
1. Gestión de Productos
2. Registro de Ventas
3. Consultar Ventas
4. Reportes Básicos
5. Análisis de Datos
6. Exportar Datos
7. Guardar Cambios
8. Salir

=== REGISTRAR VENTA ===

=== PRODUCTOS DISPONIBLES ===
   Categoría             Nombre Código  Precio Unitario  Cantidad en Stock  Precio Compra  Ganancia Unitaria  Ganancia Total Potencial
   Abarrotes              Arroz ARZ585            41.58                 66         29.106             12.474                   823.284
   Abarrotes             Azúcar AZC761            11.93                116          8.351              3.579                   415.164
   Abarrotes                Sal SAL241            34.98                 12         24.486             10.494                   125.928
   Abarrotes             Aceite ACT471            16.14                 25         11.298              4.842                   121.050
   Abarrotes             Harina HRN495            13.51                171    

<ipython-input-5-3008121231>:124: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ventas = pd.concat([ventas, nueva_venta], ignore_index=True)



Venta registrada con éxito! Total: $198.28

                 FACTURA DE VENTA                 
Fecha: 2025-06-12 22:26:36
--------------------------------------------------
Producto             Cant.    P.Unit.   Subtotal
--------------------------------------------------
Curitas                  3      35.16     105.48
Esponja                  5      18.56      92.80
--------------------------------------------------
                              TOTAL:     198.28
             ¡GRACIAS POR SU COMPRA!              


=== SISTEMA DE GESTIÓN DE TIENDA ===
1. Gestión de Productos
2. Registro de Ventas
3. Consultar Ventas
4. Reportes Básicos
5. Análisis de Datos
6. Exportar Datos
7. Guardar Cambios
8. Salir
